#Bibliotecas
##### google.colab: Específico do google collab permite acesso aos arquivos locais do usuário.

##### io: É usado para criar um buffer de memória a partir dos dados carregados pelo usuário. Esse buffer é então passado para ler o conteúdo do arquivo CSV diretamente da memória, sem a necessidade de salvar o arquivo em disco primeiro.

##### pandas: pandas: Fornece estruturas de dados para criar o DataFrame.

##### requests: Simplifica a comunicação com APIs ao permitir que você envie solicitações HTTP e processe as respostas de forma fácil .

In [ ]:
from google.colab import files
import io
import pandas as pd
import requests

# Upload

#### célula responsável pelo upload de um arquivo CSV para ser lido e manipulado para um dataframe.

In [ ]:
uploaded = files.upload()

for file_name in uploaded.keys():
    df = pd.read_csv(io.BytesIO(uploaded[file_name]))

Saving full database (2).csv to full database (2).csv


# Primeiro dataframe

#### Organiza as informaçoes obtidas do arquivo CSV em um dataframe no formato de tabela limitando as 4 primeiros moléculas.

#### Possui a função secundário de ser usado para comparar com um segundo DataFrame já com informações atualizadas.

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

df.head(4) #Seleciona o dataframe para apenas mostrar as 4 primeiras espécies.

,type,created,updated,drugbank-id,name,description,cas-number,unii,state,group,...,general-function,gene-name,theoretical-pi,molecular-weight,amino-acid-sequence,gene-sequence,average-mass,monoisotopic-mass,alternative-parent,substituent
0,biotech,2005-06-13,2024-01-02,DB00001,Lepirudin,The risk or severity of thrombocytopenia can b...,138068-37-8,Y43GF64R34,solid,withdrawn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,DB01022,Phylloquinone,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,DB01373,Calcium,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,biotech,2005-06-13,2024-01-02,DB00002,Cetuximab,The risk or severity of adverse effects can be...,205923-56-4,PQX0D8J21J,liquid,approved,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# API

#### Cria um acesso ao banco de dados Pubchem para buscar os pesos moleculares ausentes (NaN) do primeiro dataframe e os substituir por valores a serem expostos no segundo.

In [ ]:
def get_pubchem_weight(molecule_name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{molecule_name}/property/MolecularWeight/JSON"
    try:
        response = requests.get(url, timeout=30)
    except requests.exceptions.Timeout:
        print(f"Tempo limite excedido para a molécula: {molecule_name}")
        return None

    if response.status_code == 200:
        data = response.json()
        if 'PropertyTable' in data and 'Properties' in data['PropertyTable']:
            return data['PropertyTable']['Properties'][0].get('MolecularWeight')
    return None
    """
    A primeira parte do código serve para iniciar o acesso ao site descrevendo parametros de busca no site
    usando o nome da molecula para buscar o peso molecular. define a comunicação com o site e o tempo de resposta
    como segurança e controle de tempo.
    """

relatorio = []
"""
Gera um relatório dos dados que foram atualizados para verificação manual da legitimidade dos valores.
"""

for index, row in df.head(4).iterrows(): # Limita a iteração as 4 preimeiras moléculas
    if pd.isna(row['molecular-weight']):
        molecule_name = row['name']
        molecular_weight = get_pubchem_weight(molecule_name)

        if molecular_weight:
            df.at[index, 'molecular-weight'] = molecular_weight
            relatorio.append(f"Substituição na molécula '{molecule_name}': peso molecular atualizado para '{molecular_weight}'.")
        """
        Confere se o valor no peso molecular do dataframe é um NaN, caso seja verdadeiro para NaN ele será substituido para um valor
        encontrado no site pubchem. A substituição será limitada as 4 primeiras moléculas do dataframe.
        """

for item in relatorio: #Impressão do relatório
    print(item)

df.to_csv('drugbank_atualizado.csv', index=False) #Atualiza o arquivo CSV.

<ipython-input-4-8fc5bc4ac87a>:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '6979' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'molecular-weight'] = molecular_weight


Substituição na molécula 'Lepirudin': peso molecular atualizado para '6979'.
Substituição na molécula 'Phylloquinone': peso molecular atualizado para '450.7'.
Substituição na molécula 'Calcium': peso molecular atualizado para '40.08'.


# Download

#### Opção ao usuário para baixar o arquivo atualizado em CSV.


In [ ]:
down = input("Baixar CSV atualizado? Digite 's' para Sim e 'n' para Não: ").strip().lower()

if down == 's':
    files.download('drugbank_atualizado.csv')
else:
    pass

Baixar CSV atualizado? Digite 's' para Sim e 'n' para Não: n


#Segundo dataframe

#### Plota um segundo dataframe que mostra as informações atualizadas. Segue o mesmo padrão do primeiro dataframe.

#### Função secundária de comparação com o primeiro dataframe.

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()


df.head(4)

,type,created,updated,drugbank-id,name,description,cas-number,unii,state,group,...,general-function,gene-name,theoretical-pi,molecular-weight,amino-acid-sequence,gene-sequence,average-mass,monoisotopic-mass,alternative-parent,substituent
0,biotech,2005-06-13,2024-01-02,DB00001,Lepirudin,The risk or severity of thrombocytopenia can b...,138068-37-8,Y43GF64R34,solid,withdrawn,...,NaN,NaN,NaN,6979,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,DB01022,Phylloquinone,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,450.7,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,DB01373,Calcium,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,40.08,NaN,NaN,NaN,NaN,NaN,NaN
3,biotech,2005-06-13,2024-01-02,DB00002,Cetuximab,The risk or severity of adverse effects can be...,205923-56-4,PQX0D8J21J,liquid,approved,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
